In [ ]:
import os
import pprint
import string
import sys
import warnings

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import fsspec
import gcsfs
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr
import yaml
from cartopy import config
from matplotlib import cm
from tqdm.auto import tqdm
from zarr.errors import ContainsGroupError, GroupNotFoundError

sys.path.append("data_prep")
import dc6_functions

%matplotlib inline

In [ ]:
models_dict = dc6_functions.get_cmip6_models()
ensemble_members = dc6_functions.get_cmip6_ensemble_members()
grids = dc6_functions.get_cmip6_grids()
institutions = dc6_functions.get_cmip6_institutions()

In [ ]:
DC6_VERSION = "v1.1"
SUMMARY_VERSION = "v1.0"

DC6_REGION_PATT_RELEASE = (
    "https://storage.googleapis.com/downscaled-288ec5ac/diagnostics/RELEASE-{dc6_version}/"
    "21-year-average-summaries/regional-cities-release/{region}/{activity}/"
    "{institution_id}/{source_id}/{experiment_id}/"
    "{member_id}/{table_id}/{variable_id}/{summary_version}.zarr"
)

DC6_REGION_PATT_CLEAN = (
    "https://storage.googleapis.com/downscaled-288ec5ac/diagnostics/RELEASE-{dc6_version}/"
    "21-year-average-summaries/regional-cities-clean/{region}/{activity}/"
    "{institution_id}/{source_id}/{experiment_id}/"
    "{member_id}/{table_id}/{variable_id}/{summary_version}.zarr"
)

DC6_REGION_PATT_COMPARE = (
    "https://storage.googleapis.com/downscaled-288ec5ac/diagnostics/RELEASE-{dc6_version}/"
    "21-year-average-summaries/regional-cities-comparison/"
    "{region}/{source_id}/{summary_version}.zarr"
)

In [ ]:
shapefile_sources = {
    "admin1": "https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_1_states_provinces.zip",
    "admin0": "https://naciscdn.org/naturalearth/10m/cultural/ne_10m_admin_0_countries.zip",
}

In [ ]:
admin1 = gpd.read_file(shapefile_sources["admin1"])

In [ ]:
admin0 = gpd.read_file(shapefile_sources["admin0"])

In [ ]:
city_spec = pd.DataFrame(
    [
        {"city": "Tokyo", "lat": 35.681, "lon": 139.767},
        {"city": "Delhi", "lat": 28.625, "lon": 77.125},
        {"city": "Shanghai", "lat": 31.125, "lon": 121.375},
        {"city": "Sao Paulo", "lat": -23.625, "lon": -46.625},
        {"city": "Mexico City", "lat": 19.375, "lon": -99.125},
        {"city": "Cairo", "lat": 30.125, "lon": 31.125},
        {"city": "Dhaka", "lat": 23.875, "lon": 90.375},
        {"city": "New York", "lat": 40.625, "lon": -74.125},
        {"city": "Buenos Aires", "lat": -34.625, "lon": -58.375},
        {"city": "Istanbul", "lat": 41.125, "lon": 28.875},
        {"city": "Lagos", "lat": 6.510, "lon": 3.370},
        {"city": "Paris", "lat": 48.875, "lon": 2.375},
        {"city": "Moscow", "lat": 55.875, "lon": 37.625},
        {"city": "Miami", "lat": 25.875, "lon": -80.125},
        {"city": "Mumbai", "lat": 19.125, "lon": 72.875},
        {"city": "Manila", "lat": 14.599, "lon": 120.984},
        {"city": "London", "lat": 51.625, "lon": -0.125},
    ]
).set_index("city")

points = gpd.points_from_xy(city_spec.lon, city_spec.lat, crs="epsg:4326")
city_spec = gpd.GeoDataFrame(city_spec, geometry=points)
city_spec

In [ ]:
city_spec["ADM0_A3"] = gpd.sjoin(city_spec, admin0, how="left")["ADM0_A3"]
city_spec["adm1_code"] = gpd.sjoin(city_spec, admin1, how="left")["adm1_code"]

In [ ]:
city_spec

In [ ]:
shp_index_cols = {
    "admin0": "ADM0_A3",
    "admin1": "adm1_code",
}

In [ ]:
shp_meta_cols = {
    "admin0": ["NAME", "SOVEREIGNT"],
    "admin1": ["name", "adm0_a3", "admin"],
}

In [ ]:
shapes = {
    "admin0": admin0,
    "admin1": admin1,
}

In [ ]:
def cached_model_summary(source_id, region):
    fs = fsspec.filesystem(
        "gs",
        timeout=120,
        cache_timeout=120,
        requests_timeout=120,
        read_timeout=120,
        conn_timeout=120,
    )

    fs.invalidate_cache()
    fs.clear_instance_cache()

    compare_fp = DC6_REGION_PATT_COMPARE.format(
        dc6_version=DC6_VERSION,
        summary_version=SUMMARY_VERSION,
        source_id=source_id,
        region=region,
    )

    try:
        return xr.open_zarr(compare_fp, chunks=False, consolidated=True).load()
    except (IOError, GroupNotFoundError, FileNotFoundError, KeyError):
        try:
            fs.rm(
                compare_fp,
                recursive=True,
            )
            fs.invalidate_cache()
            fs.clear_instance_cache()
        except FileNotFoundError:
            pass

    data = {}

    experiments = models_dict[source_id]
    for experiment_id in experiments:
        data[experiment_id] = {}
        for variable_id in ["tasmax", "tasmin", "pr"]:
            data[experiment_id][variable_id] = {}

            activity = "CMIP" if experiment_id == "historical" else "ScenarioMIP"
            institution_id = dc6_functions.get_cmip6_institutions()[source_id]
            member_id = dc6_functions.get_cmip6_ensemble_members()[source_id]

            if (source_id == "MPI-ESM1-2-HR") and (experiment_id == "historical"):
                institution_id = "MPI-M"

            extra_kwargs = dict(
                activity=activity,
                institution_id=institution_id,
                source_id=source_id,
                experiment_id=experiment_id,
                member_id=member_id,
                table_id="21yrroll",
                variable_id=variable_id,
                dc6_version=DC6_VERSION,
                summary_version=SUMMARY_VERSION,
            )

            for stage, fpatt in [
                ("clean", DC6_REGION_PATT_CLEAN),
                ("release", DC6_REGION_PATT_RELEASE),
            ]:
                fp = fpatt.format(region=region, **extra_kwargs)
                try:
                    with xr.open_zarr(fp, chunks=None, consolidated=True) as ds:
                        ds = ds.load()
                except (KeyError, GroupNotFoundError, FileNotFoundError):
                    raise FileNotFoundError(fp)

                # bug fix in coords
                for meta_col in shp_meta_cols[region]:
                    ds.coords[meta_col] = (shp_index_cols[region],), ds[meta_col].values

                data[experiment_id][variable_id][stage] = ds

    # reorganize into a giant xr.Dataset
    ds = xr.concat(
        [
            xr.merge(
                [
                    xr.concat(
                        [
                            xr.concat(
                                [
                                    data["historical"][v]["clean"],
                                    data[scen][v]["clean"],
                                ],
                                dim="period",
                            ),
                            xr.concat(
                                [
                                    data["historical"][v]["release"],
                                    data[scen][v]["release"],
                                ],
                                dim="period",
                            ),
                        ],
                        dim=pd.Index(["clean", "release"], name="stage"),
                    )
                    for v in ["tasmin", "tasmax", "pr"]
                ]
            ).expand_dims(scen=[scen])
            for scen in models_dict[source_id][1:]
        ],
        dim="scen",
    )

    ds.attrs.update(data["historical"]["tasmax"]["release"].attrs)

    # drop attrs not relevant to this cross-file summary
    for a in [
        "activity_id",
        "experiment_id",
        "experiment",
        "variable_id",
        "variable",
        "sub_experiment",
        "sub_experiment_id",
    ]:
        ds.attrs.pop(a, None)

    for v in ["tasmin", "tasmax", "pr"]:
        for vv in data["historical"][v]["release"].data_vars.keys():
            ds[vv].attrs.update(data["historical"][v]["release"][vv].attrs)

    # clear prior encodings to handle bug in zarr spec
    # see https://github.com/pydata/xarray/issues/3476
    to_store = ds.copy()

    to_store = to_store.chunk()

    for c in to_store.coords.keys():
        if to_store.coords[c].dtype.kind in "OUS":
            to_store.coords[c] = to_store.coords[c].astype("unicode")

    for var in to_store.variables:
        to_store[var].encoding.clear()
    for var in to_store.coords.keys():
        to_store.coords[var].encoding.clear()

    try:
        to_store.to_zarr(compare_fp, consolidated=True)
    except ContainsGroupError:
        raise ContainsGroupError(compare_fp)
    except TypeError as e:
        traceback.print_exc()
        return to_store, e

In [ ]:
all_data_adm0 = {
    source_id: cached_model_summary(source_id, "admin0")
    for source_id in tqdm(models_dict.keys(), desc="admin0")
}

all_data_adm1 = {
    source_id: cached_model_summary(source_id, "admin1")
    for source_id in tqdm(models_dict.keys(), desc="admin1")
}

In [ ]:
for m in all_data_adm1.keys():
    all_data_adm1[m].coords["title"] = (
        all_data_adm1[m].coords["name"].astype("O")
        + ", "
        + all_data_adm1[m].coords["adm0_a3"].astype("O")
    )

In [ ]:
all_data_adm0["BCC-CSM2-MR"]

In [ ]:
annual_variables = [
    v
    for v in all_data_adm0["BCC-CSM2-MR"].data_vars.keys()
    if "season" not in all_data_adm0["BCC-CSM2-MR"][v].dims
]

### Scatter plots of cleaned vs. results (1994-2014 to 2089-2099) change in 21-year mean statistics for selected regions

In [ ]:
os.makedirs("outputs", exist_ok=True)

In [ ]:
correlations = {}

In [ ]:
seasonal_average_tasmax_adm0 = (
    pd.concat(
        [
            (
                all_data_adm0[m].seasonal_average_tasmax.sel(period=2089)
                - all_data_adm0[m].seasonal_average_tasmax.sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm0.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

In [ ]:
seasonal_average_tasmax_adm1 = (
    pd.concat(
        [
            (
                all_data_adm1[m].seasonal_average_tasmax.sel(period=2089)
                - all_data_adm1[m].seasonal_average_tasmax.sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

In [ ]:
title = "annual and seasonal tasmax - admin0"
print(title)
fig, axes = plt.subplots(1, 5, figsize=(14, 3), sharey=True, sharex=True)

ax = axes[0]
df = (
    pd.concat(
        [
            (
                all_data_adm0[m]["annual_average_tasmax"].sel(period=2089)
                - all_data_adm0[m]["annual_average_tasmax"].sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm0.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
)

ax.set_title("{}\n$r^2={:0.3f}$".format("annual", corr))

names = ["winter", "spring", "summer", "fall"]
for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = seasonal_average_tasmax_adm0[seasonal_average_tasmax_adm0.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(names[si], corr))

minx, maxx = min([ax.get_xlim()[0] for ax in axes]), max(
    [ax.get_xlim()[1] for ax in axes]
)
miny, maxy = min([ax.get_ylim()[0] for ax in axes]), max(
    [ax.get_ylim()[1] for ax in axes]
)
minv, maxv = min(minx, miny), max(maxx, maxy)

for ax in axes.flat:
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")
    ax.set_xticks(np.arange(0, 14, 2))
    ax.set_yticks(np.arange(0, 14, 2))
    #     ax.set_xticklabels([t.get_text() for t in axes[0].get_yticklabels()])

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")

axes[0].set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
title = "annual and seasonal tasmax - admin1"
print(title)
fig, axes = plt.subplots(1, 5, figsize=(14, 3), sharey=True, sharex=True)

ax = axes[0]
df = (
    pd.concat(
        [
            (
                all_data_adm1[m]["annual_average_tasmax"].sel(period=2089)
                - all_data_adm1[m]["annual_average_tasmax"].sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
)

ax.set_title("{}\n$r^2={:0.3f}$".format("annual", corr))

names = ["winter", "spring", "summer", "fall"]
for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = seasonal_average_tasmax_adm1[seasonal_average_tasmax_adm1.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(names[si], corr))

minx, maxx = min([ax.get_xlim()[0] for ax in axes]), max(
    [ax.get_xlim()[1] for ax in axes]
)
miny, maxy = min([ax.get_ylim()[0] for ax in axes]), max(
    [ax.get_ylim()[1] for ax in axes]
)
minv, maxv = min(minx, miny), max(maxx, maxy)

for ax in axes.flat:
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")
    ax.set_xticks(np.arange(0, 14, 2))
    ax.set_yticks(np.arange(0, 14, 2))

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")

axes[0].set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
title = "figure_7_annual_and_seasonal_tasmax_admin0_and_admin1"
print(title)
fig, axes_all = plt.subplots(2, 5, figsize=(14, 6), dpi=200, sharey=True, sharex=True)
panels = string.ascii_lowercase[:10]

# annual and seasonal tasmax - admin0
axes = axes_all[0, :]

ax = axes[0]
df = (
    pd.concat(
        [
            (
                all_data_adm0[m]["annual_average_tasmax"].sel(period=2089)
                - all_data_adm0[m]["annual_average_tasmax"].sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm0.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
    size=0.5,
    linewidth=0,
)

ax.set_title("a. {}\n$r^2={:0.3f}$".format("Annual", corr))

names = ["Dec/Jan/Feb", "Mar/Apr/May", "Jun/Jul/Aug", "Sep/Oct/Nov"]
for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = seasonal_average_tasmax_adm0[seasonal_average_tasmax_adm0.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
        size=0.5,
        linewidth=0,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}. {}\n$r^2={:0.3f}$".format(panels[si + 1], names[si], corr))

for ax in axes.flat:
    ax.set_xlabel("source GCM")

axes[0].set_ylabel("downscaled")


# annual and seasonal tasmax - admin1
axes = axes_all[1, :]

ax = axes[0]
df = (
    pd.concat(
        [
            (
                all_data_adm1[m]["annual_average_tasmax"].sel(period=2089)
                - all_data_adm1[m]["annual_average_tasmax"].sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
    size=0.5,
    linewidth=0,
)

ax.set_title("f. {}\n$r^2={:0.3f}$".format("Annual", corr))

names = ["Dec/Jan/Feb", "Mar/Apr/May", "Jun/Jul/Aug", "Sep/Oct/Nov"]
for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = seasonal_average_tasmax_adm1[seasonal_average_tasmax_adm1.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
        size=0.5,
        linewidth=0,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}. {}\n$r^2={:0.3f}$".format(panels[si + 6], names[si], corr))

for ax in axes.flat:
    ax.set_xlabel("source GCM")

axes[0].set_ylabel("downscaled")


minx, maxx = min([ax.get_xlim()[0] for ax in axes]), max(
    [ax.get_xlim()[1] for ax in axes_all.flat]
)
miny, maxy = min([ax.get_ylim()[0] for ax in axes]), max(
    [ax.get_ylim()[1] for ax in axes_all.flat]
)
minv, maxv = min(minx, miny), max(maxx, maxy)

for ax in axes_all.flat:
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")
    ax.set_xticks(np.arange(0, 14, 2))
    ax.set_yticks(np.arange(0, 14, 2))
    ax.xaxis.set_tick_params(labelbottom=True)

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")
fig.savefig(f"outputs/{title}.pdf")

In [ ]:
pprint.pprint(correlations)

In [ ]:
seasonal_average_precip_adm0 = (
    pd.concat(
        [
            (
                all_data_adm0[m].seasonal_total_precip.sel(period=2089)
                - all_data_adm0[m].seasonal_total_precip.sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm0.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

In [ ]:
seasonal_total_precip_adm1 = (
    pd.concat(
        [
            (
                all_data_adm1[m].seasonal_total_precip.sel(period=2089)
                - all_data_adm1[m].seasonal_total_precip.sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

In [ ]:
title = "annual and seasonal total precip - admin0"
print(title)
fig, axes = plt.subplots(1, 5, figsize=(14, 3), dpi=200)

ax = axes[0]
df = (
    pd.concat(
        [
            (
                all_data_adm0[m]["annual_total_precip"].sel(period=2089)
                - all_data_adm0[m]["annual_total_precip"].sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm0.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
    size=0.5,
    linewidth=0,
)

ax.set_title("{}\n$r^2={:0.3f}$".format("annual", corr))

names = ["winter", "spring", "summer", "fall"]
for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = seasonal_average_precip_adm0[seasonal_average_precip_adm0.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
        size=0.5,
        linewidth=0,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(names[si], corr))

for ax in axes.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")
    ax.set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
title = "annual and seasonal total precip - admin1"
print(title)
fig, axes = plt.subplots(1, 5, figsize=(14, 3))

ax = axes[0]
df = (
    pd.concat(
        [
            (
                all_data_adm1[m]["annual_total_precip"].sel(period=2089)
                - all_data_adm1[m]["annual_total_precip"].sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=True,
)

ax.set_title("{}\n$r^2={:0.3f}$".format("annual", corr))

names = ["winter", "spring", "summer", "fall"]

seasonal_total_precip_adm1 = (
    pd.concat(
        [
            (
                all_data_adm1[m].seasonal_total_precip.sel(period=2089)
                - all_data_adm1[m].seasonal_total_precip.sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)


for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = seasonal_total_precip_adm1[seasonal_total_precip_adm1.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(names[si], corr))

for ax in axes.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")
    ax.set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
log_seasonal_total_precip_admin0 = (
    pd.concat(
        [
            (
                np.log(all_data_adm0[m].seasonal_total_precip.sel(period=2089))
                - np.log(all_data_adm0[m].seasonal_total_precip.sel(period=2004))
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm0.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

In [ ]:
title = "log annual and seasonal total precip - admin0"
print(title)
fig, axes = plt.subplots(1, 5, figsize=(14, 3))

ax = axes[0]
df = (
    pd.concat(
        [
            (
                np.log(all_data_adm0[m]["annual_total_precip"].sel(period=2089))
                - np.log(all_data_adm0[m]["annual_total_precip"].sel(period=2004))
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
)

ax.set_title("{}\n$r^2={:0.3f}$".format("annual", corr))

names = ["winter", "spring", "summer", "fall"]

log_seasonal_total_precip_adm0 = (
    pd.concat(
        [
            (
                np.log(all_data_adm0[m].seasonal_total_precip.sel(period=2089))
                - np.log(all_data_adm0[m].seasonal_total_precip.sel(period=2004))
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)


for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = log_seasonal_total_precip_adm0[log_seasonal_total_precip_adm0.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(names[si], corr))

for ax in axes.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")
    ax.set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
title = "log annual and seasonal total precip - admin1"
print(title)
fig, axes = plt.subplots(1, 5, figsize=(14, 3))

ax = axes[0]
df = (
    pd.concat(
        [
            (
                np.log(all_data_adm1[m]["annual_total_precip"].sel(period=2089))
                - np.log(all_data_adm1[m]["annual_total_precip"].sel(period=2004))
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
)

ax.set_title("{}\n$r^2={:0.3f}$".format("annual", corr))

names = ["winter", "spring", "summer", "fall"]

log_seasonal_total_precip_adm1 = (
    pd.concat(
        [
            (
                np.log(all_data_adm1[m].seasonal_total_precip.sel(period=2089))
                - np.log(all_data_adm1[m].seasonal_total_precip.sel(period=2004))
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)


for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = log_seasonal_total_precip_adm1[log_seasonal_total_precip_adm1.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(names[si], corr))

for ax in axes.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")
    ax.set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
title = "figure_8_annual_and_seasonal_total_precip_admin0_linear_and_log"
print(title)
fig, axes_all = plt.subplots(2, 5, figsize=(14, 6), dpi=200, sharex=False, sharey=False)

names = ["Dec/Jan/Feb", "Mar/Apr/May", "Jun/Jul/Aug", "Sep/Oct/Nov"]

# annual and seasonal total precip - admin0
axes = axes_all[0, :]

ax = axes[0]
df = (
    pd.concat(
        [
            (
                all_data_adm0[m]["annual_total_precip"].sel(period=2089)
                - all_data_adm0[m]["annual_total_precip"].sel(period=2004)
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm0.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
    size=0.5,
    linewidth=0,
)

ax.set_title("a. {}\n$r^2={:0.3f}$".format("Annual", corr))

for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = seasonal_average_precip_adm0[seasonal_average_precip_adm0.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
        size=0.5,
        linewidth=0,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title(
        "{}. {}\n$r^2={:0.3f}$".format(string.ascii_lowercase[si + 1], names[si], corr)
    )

# log annual and seasonal total precip - admin0
axes = axes_all[1, :]

ax = axes[0]
df = (
    pd.concat(
        [
            (
                np.log(all_data_adm0[m]["annual_total_precip"].sel(period=2089))
                - np.log(all_data_adm0[m]["annual_total_precip"].sel(period=2004))
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)

corr = df["clean"].corr(df["release"])
correlations[title] = {"annual": corr}

sns.scatterplot(
    data=df,
    x="clean",
    y="release",
    hue="scen",
    ax=ax,
    legend=False,
    size=0.5,
    linewidth=0,
)

ax.set_title("f. {}\n$r^2={:0.3f}$".format("Annual", corr))

log_seasonal_total_precip_adm0 = (
    pd.concat(
        [
            (
                np.log(all_data_adm0[m].seasonal_total_precip.sel(period=2089))
                - np.log(all_data_adm0[m].seasonal_total_precip.sel(period=2004))
            )
            .expand_dims(source_id=[m])
            .to_series()
            .unstack("stage")
            .reset_index(drop=False)
            for m in all_data_adm1.keys()
        ]
    )
    .reset_index(drop=True)
    .rename_axis(None, axis=1)
)


for si, season in enumerate(["DJF", "MAM", "JJA", "SON"]):
    ax = axes[si + 1]

    df = log_seasonal_total_precip_adm0[log_seasonal_total_precip_adm0.season == season]

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
        size=0.5,
        linewidth=0,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][season] = corr

    ax.set_title(
        "{}. {}\n$r^2={:0.3f}$".format(string.ascii_lowercase[si + 6], names[si], corr)
    )

for ax in axes_all.flat:
    ax.set_xlabel(False)
    ax.set_ylabel(False)

for ax in axes:
    ax.set_xlabel("source GCM")

for ax in axes_all[:, 0].flat:
    ax.set_ylabel("downscaled")

for ax in axes_all.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")

for ax in axes_all.flat:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.xaxis.set_tick_params(labelbottom=True)

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")
fig.savefig(f"outputs/{title}.pdf")

# Extreme variables

In [ ]:
extremes = [
    ("days with tasmax $≥ 25{}^\circ$C", "number_of_days_with_tasmax_above_25C"),
    ("days with tasmax $≥ 90{}^\circ$F", "number_of_days_with_tasmax_above_90F"),
    ("days with tasmax $≥ 95{}^\circ$F", "number_of_days_with_tasmax_above_95F"),
    ("days with tasmin $≥ 20{}^\circ$C", "number_of_days_with_tasmin_above_20C"),
    ("days with tasmin $≥ 0{}^\circ$C", "number_of_days_with_tasmin_below_0C"),
    ("rainy days ($≥ 10$ mm)", "wetdays_above_10mm"),
    ("wet days ($≥ 1$ mm)", "wetdays_above_1mm"),
]

In [ ]:
title = "annual and seasonal extremes - admin0"
print(title)
correlations[title] = {}
fig, axes = plt.subplots(1, len(extremes), figsize=(20, 3), sharey=False, sharex=False)

for vi, (vname, variable) in enumerate(extremes):
    ax = axes[vi]

    df = (
        pd.concat(
            [
                (
                    all_data_adm0[m][variable].sel(period=2089)
                    - all_data_adm0[m][variable].sel(period=2004)
                )
                .expand_dims(source_id=[m])
                .to_series()
                .unstack("stage")
                .reset_index(drop=False)
                for m in all_data_adm0.keys()
            ]
        )
        .reset_index(drop=True)
        .rename_axis(None, axis=1)
    )

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][variable] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(vname, corr))

for ax in axes.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )

    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")
    ax.set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
title = "annual and seasonal extremes - admin1"
print(title)
correlations[title] = {}
fig, axes = plt.subplots(1, len(extremes), figsize=(20, 3), sharey=False, sharex=False)

for vi, (vname, variable) in enumerate(extremes):
    ax = axes[vi]

    df = (
        pd.concat(
            [
                (
                    all_data_adm1[m][variable].sel(period=2089)
                    - all_data_adm1[m][variable].sel(period=2004)
                )
                .expand_dims(source_id=[m])
                .to_series()
                .unstack("stage")
                .reset_index(drop=False)
                for m in all_data_adm1.keys()
            ]
        )
        .reset_index(drop=True)
        .rename_axis(None, axis=1)
    )

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][variable] = corr

    ax.set_title("{}\n$r^2={:0.3f}$".format(vname, corr))

for ax in axes.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )

    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")
    # ax.set_xticks(np.arange(0, 14, 2))
    # ax.set_yticks(np.arange(0, 14, 2))

    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)

    ax.set_xlabel("source GCM")
    ax.set_ylabel("downscaled")

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")

In [ ]:
fig, axes_all = plt.subplots(
    2, len(extremes), figsize=(20, 6), dpi=200, sharey="col", sharex="col"
)
title = "figure_9_annual_and_seasonal_extremes_admin0_and_admin1"
print(title)

# annual and seasonal extremes - admin0
axes = axes_all[0, :]
correlations[title] = {}

for vi, (vname, variable) in enumerate(extremes):
    ax = axes[vi]

    df = (
        pd.concat(
            [
                (
                    all_data_adm0[m][variable].sel(period=2089)
                    - all_data_adm0[m][variable].sel(period=2004)
                )
                .expand_dims(source_id=[m])
                .to_series()
                .unstack("stage")
                .reset_index(drop=False)
                for m in all_data_adm0.keys()
            ]
        )
        .reset_index(drop=True)
        .rename_axis(None, axis=1)
    )

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][variable] = corr

    ax.set_title(
        "{}. {}\n$r^2={:0.3f}$".format(
            string.ascii_lowercase[vi], vname[0].capitalize() + vname[1:], corr
        )
    )

# annual and seasonal extremes - admin1
axes = axes_all[1, :]

correlations[title] = {}

for vi, (vname, variable) in enumerate(extremes):
    ax = axes[vi]

    df = (
        pd.concat(
            [
                (
                    all_data_adm1[m][variable].sel(period=2089)
                    - all_data_adm1[m][variable].sel(period=2004)
                )
                .expand_dims(source_id=[m])
                .to_series()
                .unstack("stage")
                .reset_index(drop=False)
                for m in all_data_adm1.keys()
            ]
        )
        .reset_index(drop=True)
        .rename_axis(None, axis=1)
    )

    h = sns.scatterplot(
        data=df,
        x="clean",
        y="release",
        hue="scen",
        ax=ax,
        legend=False,
    )

    corr = df["clean"].corr(df["release"])
    correlations[title][variable] = corr

    ax.set_title(
        "{}. {}\n$r^2={:0.3f}$".format(
            string.ascii_lowercase[vi + len(extremes)],
            vname[0].capitalize() + vname[1:],
            corr,
        )
    )

for ax in axes_all.flat:
    ax.set_xlabel(None)
    ax.set_ylabel(None)

for ax in axes:
    ax.set_xlabel("source GCM")

for ax in axes_all[:, 0].flat:
    ax.set_ylabel("downscaled")

minv_all, maxv_all = np.inf, -np.inf

for ax in axes_all.flat:
    minv, maxv = min(ax.get_xlim()[0], ax.get_ylim()[0]), max(
        ax.get_xlim()[1], ax.get_ylim()[1]
    )
    ax.plot([minv, maxv], [minv, maxv], linestyle="dotted", zorder=-1, color="k")

for ax in axes_all.flat:
    ax.spines["top"].set_visible(False)
    ax.spines["right"].set_visible(False)
    ax.xaxis.set_tick_params(labelbottom=True)

fig.set_facecolor("white")
plt.tight_layout()
fig.savefig(f"outputs/{title}.jpg")
fig.savefig(f"outputs/{title}.pdf")

In [ ]:
corr_df = pd.concat(
    {k: pd.Series(v).rename_axis("variable") for k, v in correlations.items()},
    names=["title"],
).to_frame(name="corr")

corr_df

In [ ]:
corr_df.to_csv("outputs/correlations.csv")

# Model-specific changes across time periods

### admin 1

In [ ]:
source_id = "UKESM1-0-LL"
experiment_id = "ssp370"

assert experiment_id in models_dict[source_id], f"choose from {models_dict[source_id]}"

In [ ]:
ds = all_data_adm1[source_id].sel(scen=experiment_id)

for v in ds.data_vars.keys():
    if "season" in ds[v].dims:
        continue

    p = (
        ds[v]
        .swap_dims({"adm1_code": "title"})
        .plot.line(x="period", col="title", figsize=(30, 4))
    )
    p.fig.suptitle(f"{v}", size=16)
    p.fig.subplots_adjust(top=0.8)
    p.fig.set_facecolor("white")

for v in ds.data_vars.keys():

    if "season" not in ds[v].dims:
        continue

    for season in ds[v].season.values:
        p = (
            ds[v]
            .swap_dims({"adm1_code": "title"})
            .sel(season=season)
            .plot.line(x="period", col="title", figsize=(30, 4))
        )

        p.fig.suptitle(f"{season} {v}", size=16)

        p.fig.subplots_adjust(top=0.8)
        p.fig.set_facecolor("white")

# admin 0 

In [ ]:
ds = all_data_adm0[source_id].sel(scen=experiment_id)

for v in ds.data_vars.keys():
    if "season" in ds[v].dims:
        continue

    p = ds[v].plot.line(x="period", col="ADM0_A3", figsize=(30, 4))
    p.fig.suptitle(f"{v}", size=16)
    p.fig.subplots_adjust(top=0.8)
    p.fig.set_facecolor("white")

for v in ds.data_vars.keys():

    if "season" not in ds[v].dims:
        continue

    for season in ds[v].season.values:
        p = (
            ds[v]
            .sel(season=season)
            .plot.line(x="period", col="ADM0_A3", figsize=(30, 4))
        )

        p.fig.suptitle(f"{season} {v}", size=16)

        p.fig.subplots_adjust(top=0.8)
        p.fig.set_facecolor("white")